# CHIRPS

Aggregating CHIRPS to storm

In [3]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2

In [18]:
import ocha_stratus as stratus
import pandas as pd

from src.constants import *

## Load data

In [22]:
blob_name = f"{PROJECT_PREFIX}/processed/chirps/20240524_chirps_daily_historical_cuba.csv"
df_daily = stratus.load_csv_from_blob(blob_name, parse_dates=["date"])

In [23]:
df_daily = df_daily.sort_values("date")

In [24]:
df_daily["roll2_mean"] = df_daily["value"].rolling(2).sum()

In [25]:
df_daily

,asap0_id,name0,value,parameter,date,roll2_mean
0,35,Cuba,0.000000,precipitation,1981-01-01,NaN
1,35,Cuba,0.006035,precipitation,1981-01-02,0.006035
2,35,Cuba,0.033761,precipitation,1981-01-03,0.039797
3,35,Cuba,0.597509,precipitation,1981-01-04,0.631271
4,35,Cuba,0.744209,precipitation,1981-01-05,1.341718
...,...,...,...,...,...,...
15821,35,Cuba,1.156396,precipitation,2024-04-26,1.475423
15822,35,Cuba,0.819149,precipitation,2024-04-27,1.975545
15823,35,Cuba,0.108913,precipitation,2024-04-28,0.928062
15824,35,Cuba,3.077598,precipitation,2024-04-29,3.186511


In [26]:
blob_name = (
    f"{PROJECT_PREFIX}/processed/storm_stats/zma_stats_imerg_quantiles.parquet"
)

df_stats = stratus.load_parquet_from_blob(blob_name)

In [27]:
df_stats

,sid,valid_time_min,valid_time_max,wind_speed_max,wind_speed_max_landfall,q50_total,q50_total_mean_abv,q50_roll2,q50_roll2_mean_abv,q50_roll3,...,q95_roll2,q95_roll2_mean_abv,q95_roll3,q95_roll3_mean_abv,q99_total,q99_total_mean_abv,q99_roll2,q99_roll2_mean_abv,q99_roll3,q99_roll3_mean_abv
0,2000233N12316,2000-08-24 00:00:00.000040,2000-08-24 12:00:00.000040,40.0,NaN,18.190000,35.262634,18.152500,34.911701,18.189999,...,53.436249,60.977757,54.452000,61.702854,66.406698,70.882492,66.313896,70.659996,66.406700,70.882492
1,2000260N15308,2000-09-19 12:00:00.000040,2000-09-20 18:00:00.000040,30.0,NaN,26.412498,68.799995,21.250000,62.718685,23.750000,...,126.502251,148.864792,129.048248,153.011627,173.678444,184.858002,167.547638,181.145996,173.454788,184.656494
2,2001303N13276,2001-11-04 06:00:00.000040,2001-11-05 06:00:00.000040,120.0,115.0,94.369999,214.020828,76.807495,195.712280,94.297501,...,316.773743,356.677979,331.768250,366.694794,390.244804,404.398499,379.588867,395.611481,390.244812,404.398499
3,2002258N10300,2002-09-19 18:00:00.000040,2002-09-21 18:00:00.000040,110.0,75.0,78.592495,207.540146,40.644997,143.821213,56.139999,...,390.434479,552.790833,430.520508,613.966919,737.002824,813.606506,650.347717,694.172974,724.066223,794.184509
4,2002265N10315,2002-09-29 00:00:00.000040,2002-10-02 00:00:00.000040,90.0,90.0,54.195000,89.869759,37.119995,67.565186,43.507500,...,112.689499,134.278168,125.203247,143.808563,193.806806,215.481491,150.692581,176.249969,160.742477,183.679001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,2023321N15278,2023-11-18 00:00:00.000039,2023-11-18 00:00:00.000039,30.0,NaN,3.947500,38.300976,3.947500,38.281227,3.947500,...,115.891747,137.600708,116.012749,137.641006,154.484043,161.707001,154.373993,161.707001,154.484039,161.707001
58,2024216N20284,2024-08-02 12:00:00.000039,2024-08-04 00:00:00.000039,35.0,30.0,54.164997,103.377747,40.412502,75.217384,49.295002,...,150.748505,189.861526,156.888000,196.923157,232.989563,251.388504,215.226379,233.986496,222.375443,240.353485
59,2024268N17278,2024-09-25 12:00:00.000039,2024-09-25 12:00:00.000039,65.0,NaN,25.417499,116.488297,21.440001,99.156189,25.417500,...,189.857498,225.405807,227.883011,262.529694,282.273999,307.214508,246.513580,265.378998,282.273987,307.214508
60,2024293N21294,2024-10-20 06:00:00.000039,2024-10-22 06:00:00.000039,75.0,75.0,12.205000,61.957958,9.135000,50.185165,12.029999,...,129.257507,277.156830,148.582474,296.754669,439.478544,537.151001,427.730560,509.317963,435.983795,533.183960


## Aggregate to storm

In [28]:
def get_storm_rainfall(storm_row):
    min_date = storm_row["valid_time_min"].date()
    max_date = storm_row["valid_time_max"].date() + pd.DateOffset(days=1)
    dff_imerg = df_daily[
        (df_daily["date"] >= pd.Timestamp(min_date))
        & (df_daily["date"] <= pd.Timestamp(max_date))
    ]
    storm_row["chirps_roll2_mean"] = dff_imerg["roll2_mean"].max()
    return storm_row

In [30]:
df_stats = df_stats.apply(get_storm_rainfall, axis=1)

## Save

In [36]:
blob_name = f"{PROJECT_PREFIX}/processed/chirps/chirps_stats.parquet"
df_stats[["sid", "chirps_roll2_mean"]]
stratus.upload_parquet_to_blob(
    df_stats[["sid", "chirps_roll2_mean"]], blob_name
)